In [38]:
from pymongo import MongoClient
import pandas as pd
import os
import pathlib
from dotenv import load_dotenv
import datetime
from fredapi import Fred

env_path = pathlib.Path('..') / '.env'
load_dotenv(dotenv_path=env_path)
FRED_API_KEY = os.getenv("FRED_API_KEY")
fred = Fred(api_key=FRED_API_KEY)


client = MongoClient('localhost', 27017)
db = client.project2_db
collection = db.project2_collection

# Define your series dictionary, start and end dates as before
series_dict = {
    'bonds2tr': ('DGS2', 'D'),
    'gdpworld': ('NYGDPMKTPCDWLD', 'A'),
    'gdp': ('GDP', 'Q'),
    'recession': ('JHDUSRGDPBR', 'Q'),
    'bonds10tr': ('DGS10', 'D'),
    'ppi': ('PPIACO', 'M'),
    'cpi': ('CPIAUCSL', 'M'),
    'unrate': ('UNRATE', 'M'),
    'debt': ('GFDEBTN', 'Q'),
    'fedrate': ('FEDFUNDS', 'M'),
    'm0': ('BOGMBASE', 'M'),
    'm3': ('MABMM301USM189S', 'M'),
    'cbasstogdp': ('DDDI06USA156NWDB', 'A'),
    'resins': ('TOTRESNS', 'M'),
    'oil': ('WTISPLC', 'M'),
    'indpro': ('INDPRO', 'M'),
    'houses': ('MSPUS', 'Q'),
    'wages': ('AHETPI', 'M'),
    'cp': ('CP', 'Q'),
    'fed_debt_to_gdp': ('GFDGDPA188S', 'A'),
    'comm_banks_borrowings': ('LOANINV', 'D')
}

# Initialize DataFrames
dfs = {'D': pd.DataFrame(), 'M': pd.DataFrame(), 'Q': pd.DataFrame(), 'A': pd.DataFrame()}


def fetch_data(df, var_name, series_id):
    series_data = fred.get_series(series_id)
    series_df = series_data.to_frame(name=var_name)
    series_df.index.name = 'date'
    return df.join(series_df, how='outer') if not df.empty else series_df


for var_name, (series_id, freq) in series_dict.items():
    dfs[freq] = fetch_data(dfs[freq], var_name, series_id)

dfs

{'D':             bonds2tr  bonds10tr  comm_banks_borrowings
 date                                                  
 1962-01-02       NaN       4.06                    NaN
 1962-01-03       NaN       4.03                    NaN
 1962-01-04       NaN       3.99                    NaN
 1962-01-05       NaN       4.02                    NaN
 1962-01-08       NaN       4.03                    NaN
 ...              ...        ...                    ...
 2024-05-24      4.93       4.46                    NaN
 2024-05-27       NaN        NaN                    NaN
 2024-05-28      4.94       4.54                    NaN
 2024-05-29      4.96       4.61                    NaN
 2024-05-30      4.92       4.55                    NaN
 
 [16283 rows x 3 columns],
 'M':                 ppi      cpi  unrate  fedrate         m0  m3  resins    oil   
 date                                                                          
 1913-01-01   12.100      NaN     NaN      NaN        NaN NaN     NaN    

In [39]:
df = {}
df['D_M'] = dfs['D'].resample('M').last()
df['M_M'] = dfs['M'].resample('M').last()
df['Q_M'] = dfs['Q'].resample('M').ffill()
# new_dates = pd.date_range(
#     start=df['Q_M'].index.min(), end='2022-12-31', freq='M')
# df['Q_M'] = df['Q_M'].reindex(new_dates).ffill()
df['A_M'] = dfs['A'].resample('M').ffill()
df

{'D_M':             bonds2tr  bonds10tr  comm_banks_borrowings
 date                                                  
 1962-01-31       NaN       4.10                    NaN
 1962-02-28       NaN       4.00                    NaN
 1962-03-31       NaN       3.86                    NaN
 1962-04-30       NaN       3.86                    NaN
 1962-05-31       NaN       3.90                    NaN
 ...              ...        ...                    ...
 2024-01-31      4.27       3.99              2331822.9
 2024-02-29      4.64       4.25              2353771.8
 2024-03-31      4.59       4.20              2345546.4
 2024-04-30      5.04       4.69              2344863.7
 2024-05-31      4.92       4.55              2338338.2
 
 [749 rows x 3 columns],
 'M_M':                 ppi      cpi  unrate  fedrate         m0  m3  resins    oil   
 date                                                                          
 1913-01-31   12.100      NaN     NaN      NaN        NaN NaN     NaN  

In [42]:
data = pd.concat([df['D_M'], df['M_M'], df['Q_M'], df['A_M']], axis=1)
# data = data.loc['1970-01-01':]
# data.head(5)
data['gdp']

date
1913-01-31          NaN
1913-02-28          NaN
1913-03-31          NaN
1913-04-30          NaN
1913-05-31          NaN
                ...    
2024-01-31    28255.928
2024-02-29          NaN
2024-03-31          NaN
2024-04-30          NaN
2024-05-31          NaN
Freq: M, Name: gdp, Length: 1337, dtype: float64

In [60]:
data['wages_month'] = round(data['wages'] * 168, 2)
data['house_wages'] = round(data['houses']/data['wages_month'], 2)
data['iyc'] = round(data['bonds10tr'] - data['bonds2tr'], 2)
data['gdp_pct'] = round(data['gdp'].pct_change(periods=4) * 100, 2)
data['gdp_pct_ma4'] = round(data['gdp_pct'].rolling(window=4).mean(), 2)
data['gdpworld_pct'] = round(data['gdpworld'].pct_change(12) * 100, 2)
data['debt_to_gdp'] = round(100 * data['debt'] / data['gdp']/1000, 2)
data['resins_to_gdp'] = round(100 * data['resins'] / data['gdp'], 2)
data['cp_to_gdp'] = round(100 * data['cp'] / data['gdp'], 2)
data['m0_to_gdp'] = round(data['m0'] / data['gdp']/1000 * 100, 2)
data['m3_to_gdp'] = round((data['m3']/1000000) / data['gdp']/1000 * 100, 2)
data['cpi_pct'] = round(data['cpi'].pct_change(periods=12) * 100, 2)
data['ppi_pct'] = round(data['ppi'].pct_change(periods=12) * 100, 2)
data['cbasstogdp'] = round(data['cbasstogdp'], 2)
data['indpro'] = round(data['indpro'], 2)
data['oil'] = round(data['oil'], 2)
data['comm_banks_to_gdp'] = round(100 * (data['comm_banks_borrowings'] / (1000 * data['gdp'] )),2)
# 01.2020
base_year = 258.906
data['adjusted_oil'] = data['oil'] * (base_year / data['cpi'])
data['adjusted_oil']
data

,bonds2tr,bonds10tr,comm_banks_borrowings,ppi,cpi,unrate,fedrate,m0,m3,resins,...,cp_to_gdp,m0_to_gdp,m3_to_gdp,cpi_pct,ppi_pct,comm_banks_to_gdp,cpi_multiplier,cumulative_cpi,real_oil_price,adjusted_oil
date,,,,,,,,,,,,,,,,,,,,,
1913-01-31,NaN,NaN,NaN,12.100,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-02-28,NaN,NaN,NaN,12.000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-03-31,NaN,NaN,NaN,12.000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-04-30,NaN,NaN,NaN,12.000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-05-31,NaN,NaN,NaN,11.900,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-31,4.27,3.99,2331822.9,251.245,309.685,3.7,5.33,5843700.0,NaN,3507.0,...,11.31,20.68,NaN,3.11,-3.45,8.25,4.11,NaN,NaN,61.991636
2024-02-29,4.64,4.25,2353771.8,254.840,311.054,3.9,5.33,5896900.0,NaN,3567.7,...,NaN,NaN,NaN,3.17,-1.48,NaN,4.17,NaN,NaN,64.299088
2024-03-31,4.59,4.20,2345546.4,254.995,312.230,3.8,5.33,5883000.0,NaN,3543.1,...,NaN,NaN,NaN,3.48,-0.80,NaN,4.48,NaN,NaN,67.398647


In [61]:
data.index.name = 'date'
data = data.reset_index()

data_to_insert = data.to_dict(orient='records')
collection.insert_many(data_to_insert)

In [53]:
# cols_to_save = ['oil', 'cpi', 'cpi_pct']
# data[cols_to_save].to_csv('out.csv', index=True)
df = data

date
1913-01-31          NaN
1913-02-28          NaN
1913-03-31          NaN
1913-04-30          NaN
1913-05-31          NaN
                ...    
2024-01-31    61.991636
2024-02-29    64.299088
2024-03-31    67.398647
2024-04-30    70.552788
2024-05-31          NaN
Freq: M, Name: adjusted_oil, Length: 1337, dtype: float64